In [1]:
# this is our preamble cell :
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib 


import category_encoders as ce
from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

from sklearn import cluster
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree

# from shap import TreeExplainer, initjs, force_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot

In [7]:
# importing and minor cleaning first, parsing dates. 

dfreal = pd.read_csv('True.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')
dfreal['Fake'] = 0
print('Real data shape: ', dfreal.shape)

dffake = pd.read_csv('Fake.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')

dffake['Fake'] = 1

# I have added the boolean column for Fake/Real to each. 

# This next bit will clean up the messy date columns from the fake csv. 

searchfor = ['http', '-', 'MSNBC']
dffake = dffake[~dffake['date'].str.contains('|'.join(searchfor))]

print('Fake data shape: ', dffake.shape)

Real data shape:  (21417, 5)
Fake data shape:  (23436, 5)


In [8]:
# Now, I want to trim them to be the same size, so that we have our baseline right at 0.5 -the same as flipping a coin. 

dfreal_trimmed = dfreal[-21_400 :]
print('Real trimmed shape: ', dfreal_trimmed.shape)

dffake_trimmed = dffake[-21_400 :]
print('Fake trimmed shape: ', dffake_trimmed.shape)

# and now combine them into one dataframe:
df_joined = dfreal_trimmed.append(dffake_trimmed, ignore_index=True)

df_joined['date'] = pd.to_datetime(df_joined['date'])

print()
print('Combined and trimmed (equal parts Real and Fake) shape: ', df_joined.shape)
print()
print(df_joined.info)

Real trimmed shape:  (21400, 5)
Fake trimmed shape:  (21400, 5)

Combined and trimmed (equal parts Real and Fake) shape:  (42800, 5)

<bound method DataFrame.info of                                                    title  \
0      Trump on Twitter (Dec 26) - Hillary Clinton, T...   
1      U.S. appeals court rejects challenge to Trump ...   
2      Treasury Secretary Mnuchin was sent gift-wrapp...   
3      Federal judge partially lifts Trump's latest r...   
4      Exclusive: U.S. memo weakens guidelines for pr...   
...                                                  ...   
42795  McPain: John McCain Furious That Iran Treated ...   
42796  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
42797  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
42798  How to Blow $700 Million: Al Jazeera America F...   
42799  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text       subject  \
0      The following statements were po

In [9]:
# we're going to need to drop those colums that will interfere with the model once we have vectorized the text. 
# for now, we will drop subject and leave date. 

df_joined.drop(['subject'], axis=1, inplace=True)
print(df_joined.shape)
print(df_joined.columns)

(42800, 4)
Index(['title', 'text', 'date', 'Fake'], dtype='object')


In [ ]:
# now I'm going to create another frame for when i want to vectorize the titles. 